In [36]:
import pandas as pd

#Import data
data = pd.read_csv("shelter-animal-outcomes/train.csv.gz")

#Look at data
#data.head(10)
#data.describe()

#Cleaning

#Drop NAs
data = data.drop(['OutcomeSubtype', 'Name'],axis=1).dropna()

#Code Age
data['AgeNumeric']=data['AgeuponOutcome'].str.split(" ", n = 1, expand = True)[0].astype('float')
data['AgeString']=data['AgeuponOutcome'].str.split(" ", n = 1, expand = True)[1]
data.loc[data['AgeString'].str[0]=='y' , 'AgeInDays'] = data['AgeNumeric']*365.25
data.loc[data['AgeString'].str[0]=='w' , 'AgeInDays'] = data['AgeNumeric']*7
data.loc[data['AgeString'].str[0]=='m' , 'AgeInDays'] = data['AgeNumeric']*(365.25/12)
data.loc[data['AgeString'].str[0]=='d' , 'AgeInDays'] = data['AgeNumeric']


# Code state and sex 
data["Intact"] = data["SexuponOutcome"].str.split(" ", n = 1, expand = True)[0] 
data["Sex"]= data["SexuponOutcome"].str.split(" ", n = 1, expand = True)[1]

#Date time stuff
data['date_year'] = data['DateTime'].str[0:4]
data['date_month'] = data['DateTime'].str[5:7]
data['date_day'] = data['DateTime'].str[8:10]

#Color

#Get top 10 colors
data["Color"] = data["Color"].str.replace('/',' ')
color_freq = data["Color"].str.split(expand=True).stack().value_counts()[:10]
#print(color_freq)

#Create top 10 color cols
for i in range(0 , color_freq.count()):
    data.loc[data["Color"].str.find(color_freq.index[i])>-1,color_freq.index[i]] = 1
    data[color_freq.index[i]].fillna(0, inplace=True)

#Breed
    
#Get top 10 breed
data["Breed"] = data["Breed"].str.replace('/',' ')
breed_freq = data["Breed"].str.split(expand=True).stack().value_counts()[:10]
#print(breed_freq)

#Create top 10 breed cols
for i in range(0 , breed_freq.count()):
    data.loc[data["Breed"].str.find(breed_freq.index[i])>-1,breed_freq.index[i]] = 1
    #print(breed_freq.index[i])
    data[breed_freq.index[i]].fillna(0, inplace=True)    

#Drop unnecessary vars
#data = data.drop(['SexuponOutcome','AgeuponOutcome','AgeNumeric','AgeString','Color','DateTime'],axis=1)

data.head(50)




,AnimalID,DateTime,OutcomeType,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,AgeNumeric,AgeString,...,Mix,Shorthair,Domestic,Chihuahua,Retriever,Bull,Pit,Labrador,Terrier,Shepherd
0,A671945,2014-02-12 18:22:00,Return_to_owner,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown White,1.0,year,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A656520,2013-10-13 12:44:00,Euthanasia,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby,1.0,year,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A686464,2015-01-31 12:28:00,Adoption,Dog,Neutered Male,2 years,Pit Bull Mix,Blue White,2.0,years,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,A683430,2014-07-11 19:09:00,Transfer,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream,3.0,weeks,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,A667013,2013-11-15 12:52:00,Transfer,Dog,Neutered Male,2 years,Lhasa Apso Miniature Poodle,Tan,2.0,years,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,A677334,2014-04-25 13:04:00,Transfer,Dog,Intact Female,1 month,Cairn Terrier Chihuahua Shorthair,Black Tan,1.0,month,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
6,A699218,2015-03-28 13:11:00,Transfer,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Tabby,3.0,weeks,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,A701489,2015-04-30 17:02:00,Transfer,Cat,Unknown,3 weeks,Domestic Shorthair Mix,Brown Tabby,3.0,weeks,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,A671784,2014-02-04 17:17:00,Adoption,Dog,Spayed Female,5 months,American Pit Bull Terrier Mix,Red White,5.0,months,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
9,A677747,2014-05-03 07:48:00,Adoption,Dog,Spayed Female,1 year,Cairn Terrier,White,1.0,year,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
